In [1]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64

stride_conv1 = 1
stride_conv2 = 1

k_conv1 = 5
k_conv2 = 5

k_max_pool1 = 2
k_max_pool2 = 2

n_hidden = 1024
n_output = 10

input_size_to_hidden = (input_width//(k_max_pool1 * k_max_pool2))* (input_height//(k_max_pool1 * k_max_pool2)) * n_conv2

In [5]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([k_conv1, k_conv1, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([k_conv2, k_conv2, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo"  : tf.Variable(tf.random_normal([n_hidden, n_output])) 
}
biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo"  : tf.Variable(tf.random_normal([n_output]))
}

## Forward Propagation

In [6]:
def conv(x, weights, bias, strides = 1):
    #Apply the filter on image
    out = tf.nn.conv2d(x, weights, padding="SAME", strides= [1, strides, strides, 1])
    
    #Adding biases
    out = tf.nn.bias_add(out, bias)
    
    #Applying relu activation function
    out = tf.nn.relu(out)
    
    return out

def maxPooling(x, k=2):
    return tf.nn.max_pool(x, padding ="SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [7]:
def cnn(x, weights, biases, keep_prob):
    # Reshape the image
    x = tf.reshape(x, shape = [-1 , input_width, input_height, input_channels])
    
    # Passing it with the CNN 1
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxPooling(conv1, k_max_pool1)
    
    # Passing it with the CNN 2
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxPooling(conv2, k_max_pool2)
    
    #Reshape the image
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    
    #Passing it with hidden dense layer
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1']) 
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    
    # Passing it with Dropout Layer
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob)
    
    #Passing it with output layer
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    
    return output

Note: we need to make these changes in our code if we change the padding in conv & maxpool to “valid:

    1. Weights of the hidden layer
    2. Reshaping code before feeding into Hidden layer 

In [9]:
# Making Predictions by forward Propagation
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32,[None, n_output] )
keep_prob = tf.placeholder("float")

pred = cnn(x, weights, biases, keep_prob)

In [10]:
#Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

In [11]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [12]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
batch_size = 100
# Running multiple iterations of Optimizer
for iter in range(25):
    # Batch Gradient Descent
    num_batches = int(mnist.train.num_examples / batch_size)
    total_cost = 0  #Total cost in a batch
    for j in range(num_batches):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimize], feed_dict = {x : batch_x, y: batch_y, keep_prob:0.8})
        total_cost += c
    print(total_cost)

1124996.3135261536
39610.884020231664
23037.08407678287
15854.257151158381
11993.914579317989
8614.258532516846
7965.49903286912
8661.198405948468
7381.678427191073
6265.236901548046
4851.742510277588
5768.2143898897275
5588.808284191156
5024.905583983153
3718.332969117444
3456.853692193401
3324.0945233419234
3709.0158968678993
3111.8419402707123
3053.3275695438642
2776.7635871437988
2687.196164058223
2775.540983928223
1867.5103327859388
1915.9476898354976


In [14]:
predictions = tf.argmax(pred, 1)  
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)

predictions, correct_predictions =  sess.run([predictions, correct_predictions], 
                                                   feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0})
correct_predictions.sum()

9860

Without dropout : 9859

With droupout : 9860 